In [3]:
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
# [Default]

def OS_Driver(os,browser):
    if os.lower() == 'windows':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS('C:/Users/pc/Documents/phantomjs-2.1.1-window/bin/phantomjs.exe')
        else:
            driver = webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')
    elif os.lower() == 'mac':
        if browser.lowser() == 'phantom':
            driver = webdriver.PhantomJS(
                '/Users/hyunyoun/Documents/GitHub/Private_Project/phantomjs-2.1.1/bin/phantomjs')
        else:
            driver = webdriver.Chrome('../chromedriver')
    return driver
def OpenAPI(apiFile):
    apiKey = pickle.load(open(apiFile, 'rb'))
    return apiKey
## Define Error
class NotMatch(Exception):
    def __init__(self, f):
        self.f = f
    def __str__(self):
        return self.f
## Setting BeautifulSoup
def Setting_BeautifulSoup(parser):
    if parser.lower() == 'xml':
        parser = 'lxml'
    elif parser.lower() == 'html':
        parser = 'html.parser'
    else:
        raise NotMatch('Not Match parser')
    return parser
## Using BeautifulSoup
def WebPageUsingBS(url, htmltype, openApi=None):
    htmltype = Setting_BeautifulSoup(htmltype)
    if openApi == None:
        response = requests.get(url)
    else:
        response = requests.get(url, headers=openApi)
    soup = BeautifulSoup(response.content, htmltype)
    return soup

In [7]:
driver = webdriver.Chrome('../../../chromedriver.exe')

In [13]:
w2 = 'http://v.media.daum.net/v/20171202023910089'

In [14]:
driver.get(w2)

In [15]:
try:
    element = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'alex-area')))
except:pass
else:
    loop = True
    while loop:
        try:
            commentsByclass = 'alex_more'
            element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME, commentsByclass)))
            more_button = driver.find_element_by_class_name(commentsByclass)
            webdriver.ActionChains(driver).move_to_element(more_button).click(more_button).perform()
            commentElements = driver.find_elements_by_class_name('cmt_info')
        except:
            loop = False

In [18]:
def CommentsInDaum(df):
    df[r'공감'] = re.search('[\d]+', df[r'공감']).group()
    df[r'비공감'] = re.search('[\d]+', df[r'비공감']).group()
    if df['comments'] == '':
        comments = '오직 이모티콘만 있는 댓글입니다.'
    else:
        comments = ' '.join(df['comments'].split('\n'))
    df['comments'] = comments
    return df

In [19]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
comments= soup.select('#alex-area > div > div > div > div.cmt_box > ul.list_comment > li > div.cmt_info')

In [20]:
xx = list(map(lambda x: (x.find('p', class_ = 'desc_txt').text, x.find('button',class_ = '#like').text, x.find('button',class_ = '#dislike').text), comments))
xx = pd.DataFrame(xx)

AttributeError: 'NoneType' object has no attribute 'text'

In [21]:
for i in comments:
    try:
        i.find('p',class_='desc_txt').text
        i.find('button',class_ = '#like').text
        i.find('button',class_ = '#dislike').text
    except:
        print (i)
        break

<div class="cmt_info" data-reactid=".0.0.0.4.1.$194300815.0"><strong class="tit_nick" data-reactid=".0.0.0.4.1.$194300815.0.0"><span class="info_author" data-reactid=".0.0.0.4.1.$194300815.0.0.0"><span class="link_nick" data-reactid=".0.0.0.4.1.$194300815.0.0.0.0"><span class="img_daum" data-reactid=".0.0.0.4.1.$194300815.0.0.0.0.0"></span></span><a class="link_nick" data-reactid=".0.0.0.4.1.$194300815.0.0.0.1">탈퇴한 사용자</a><span class="txt_date" data-reactid=".0.0.0.4.1.$194300815.0.0.0.2">2017.12.02 12:25</span></span></strong><p class="desc_txt font_size_17" data-reactid=".0.0.0.4.1.$194300815.0.2"><em class="emph_txt" data-reactid=".0.0.0.4.1.$194300815.0.2.0">삭제</em><span data-reactid=".0.0.0.4.1.$194300815.0.2.1">된 글입니다.</span></p></div>
